Name: Robert (Bobby) Fisher
CWID: 11187333

In [1]:
# import required modules
from pyspark.sql.functions import udf
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import HashingTF
from pyspark.sql.functions import udf
from pyspark.sql.functions import when
from pyspark.sql.types import StringType
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import re


In [2]:
# load sms data into a DataFrame
sms_df = spark.read.format("csv") \
    .option("header", "true") \
    .load('hdfs://localhost:9000/user/pistolpete/spark_data/sms.data')

In [3]:
# function to remove non-alpha-numeric characters
def removeNonAlphaNum(x):
    y = re.sub(r'[^\w\s]', '', x)
    z = y.lower()
    return z

# create a user-defined function and use through Spark SQL
removePunctuationUDF = udf(removeNonAlphaNum, StringType()) 
sms_df_1 = sms_df.select(
    when(sms_df["class"] == "spam", 1).otherwise(0).alias("class"), 
    removePunctuationUDF(sms_df['text']).alias("text"))

# examine new schema
sms_df_1.printSchema()

root
 |-- class: integer (nullable = false)
 |-- text: string (nullable = true)



In [4]:
# Split DataFrame into training and testing parts
df_train, df_test = sms_df_1.randomSplit([0.8, 0.2], seed = 42)
print('Train: ', df_train.count())
print('Test : ', df_test.count())

Train:  4501
Test :  1073


In [5]:
# Create Feature transformer and random forest classifier objects 
tk_obj = Tokenizer(inputCol = "text", outputCol = "tokens")
swr_obj = StopWordsRemover(inputCol = "tokens", outputCol = "filtered")
htf_obj = HashingTF(numFeatures = 25, inputCol = "filtered", outputCol = "features")
rf_obj = RandomForestClassifier(labelCol = "class", featuresCol = "features", numTrees = 50)

# Create pipeline object and fit on training data
pl_obj = Pipeline(stages = [tk_obj, swr_obj, htf_obj, rf_obj])
pl_model = pl_obj.fit(df_train)

In [6]:
# Score testing data
pl_pred = pl_model.transform(df_test)
pl_pred_1 = pl_pred.select(
    when(pl_pred["class"] == 1, "spam").otherwise("ham").alias("class"), 
    when(pl_pred["prediction"] == 1, "spam").otherwise("ham").alias("prediction"))

In [7]:
# Function to compute Accuracy
def Accuracy(df, orig_col = "class", pred_col = "prediction"):
    n = df.count()
    n_match = df.filter(df[orig_col] == df[pred_col]).count()
    return n_match * 100.0 / n

# Compute and print accuracy and confusion matrix
pl_acc_1 = Accuracy(pl_pred_1)
print("Accuracy : ", pl_acc_1)
print("Confusion Matrix: ")
pl_pred_1.crosstab("class", "prediction").show()

Accuracy :  87.88443616029824
Confusion Matrix: 
+----------------+---+----+
|class_prediction|ham|spam|
+----------------+---+----+
|            spam|128|  24|
|             ham|919|   2|
+----------------+---+----+



In [8]:
# Create ParamGrid object; 
# Represents various combinations to be evaluated
paramGrid = ParamGridBuilder() \
        .addGrid(htf_obj.numFeatures, [25, 250, 250]) \
        .addGrid(rf_obj.numTrees, [50, 100, 200]) \
        .build()
        
# Create cross-validator object
cv_obj = CrossValidator(
    estimator = pl_obj,
    estimatorParamMaps = paramGrid,
    evaluator = BinaryClassificationEvaluator(labelCol = "class"),
    numFolds = 5)

In [9]:
# Find best model using training data and cross-validator object
cv_model = cv_obj.fit(df_train)

# Score testing data
pred_cv = cv_model.transform(df_test)
pred_cv_1 = pred_cv.select(
    when(pred_cv["class"] == 1, "spam").otherwise("ham").alias("class"), 
    when(pred_cv["prediction"] == 1, "spam").otherwise("ham").alias("prediction"))

In [10]:
# Compute and print accuracy and confusion matrix
cv_acc_1 = Accuracy(pred_cv_1)
print("Accuracy : ", cv_acc_1)
print("Confusion Matrix: ")
pred_cv_1.crosstab("class", "prediction").show()

Accuracy :  88.0708294501398
Confusion Matrix: 
+----------------+---+----+
|class_prediction|ham|spam|
+----------------+---+----+
|            spam|128|  24|
|             ham|921|   0|
+----------------+---+----+

